In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import scipy.stats as stats
data = r"C:\Users\Lenovo\Documents\Data"


In [2]:
#UDF 
def fn_descriptives( x ):
    
    n_tot = x.isnull().count()
    n_miss = x.isnull().sum()
    n_miss_perc = n_miss / n_tot
    
    p25 = x.quantile(0.25)
    p75 = x.quantile(0.75)
    
    iqr = p75 - p25
    
    lc_iqr = p25 - 1.5 * iqr
    uc_iqr = p75 + 1.5 * iqr
    
    return pd.Series( [ x.dtype, x.nunique(), n_tot, x.count(), n_miss, n_miss_perc, 
                       x.sum(), x.var(), x.std(), x.mean(), 
                       iqr, lc_iqr, uc_iqr, 
                       x.min(), x.quantile(0.01), x.quantile(0.05), x.quantile(0.10), 
                       x.quantile(0.25), x.quantile(0.50), x.quantile(0.75), 
                       x.quantile(0.90), x.quantile(0.95), x.quantile(0.99), x.max()],
                     
                    index = ['dtype', 'cardinality', 'n_tot', 'n', 'n_miss', 'n_miss_perc',
                             'sum', 'var', 'std', 'mean',
                             'iqr', 'lc_iqr', 'uc_iqr',
                             'min', 'p1', 'p5', 'p10', 
                             'p25', 'p50', 'p75', 
                             'p90', 'p95', 'p99', 'max'])

## BUSINESS PROBLEM-1

In [3]:
loan_data = pd.read_csv(data + '\\LoansData.csv')
loan_data.head(2)

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,20000.0,20000.0,8.90%,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14.0,14272.0,2.0,< 1 year
1,19200.0,19200.0,12.12%,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12.0,11140.0,1.0,2 years


#### DATA PREP

In [4]:
loan_data.shape

(2500, 14)

In [5]:
loan_data.columns

Index(['Amount.Requested', 'Amount.Funded.By.Investors', 'Interest.Rate',
       'Loan.Length', 'Loan.Purpose', 'Debt.To.Income.Ratio', 'State',
       'Home.Ownership', 'Monthly.Income', 'FICO.Range', 'Open.CREDIT.Lines',
       'Revolving.CREDIT.Balance', 'Inquiries.in.the.Last.6.Months',
       'Employment.Length'],
      dtype='object')

In [6]:
#Assigning the correct column names of the data
loan_data.rename( {'Amount.Requested': 'amt_req', 'Amount.Funded.By.Investors': 'amt_funded_by_investors', 'Interest.Rate': 'interest_rate',
       'Loan.Length': 'loan_length_Month', 'Loan.Purpose': 'loan_purpose', 'Debt.To.Income.Ratio': 'debt_to_inc_ratio',
       'Home.Ownership': 'home_ownership', 'Monthly.Income':'monthly_inc', 'FICO.Range':'FICO_Range', 'Open.CREDIT.Lines':'open_REDIT_lines',
       'Revolving.CREDIT.Balance': 'revolving_CREDIT_Bal' , 'Inquiries.in.the.Last.6.Months':'last_6month_inquiries',
       'Employment.Length':'employment_Length'}, axis= 1 , inplace= True)

In [7]:
loan_data.columns

Index(['amt_req', 'amt_funded_by_investors', 'interest_rate',
       'loan_length_Month', 'loan_purpose', 'debt_to_inc_ratio', 'State',
       'home_ownership', 'monthly_inc', 'FICO_Range', 'open_REDIT_lines',
       'revolving_CREDIT_Bal', 'last_6month_inquiries', 'employment_Length'],
      dtype='object')

In [8]:
# removing % symbol from the data
loan_data.interest_rate = loan_data.interest_rate.str.replace("%", '').astype('float')
loan_data.debt_to_inc_ratio = loan_data.debt_to_inc_ratio.str.replace("%", '').astype('float')
# loan length col
loan_data.loan_length_Month = loan_data.loan_length_Month.str.replace("months", '').astype('float')

In [9]:
# null value treatment
loan_data.isnull().sum()

amt_req                     1
amt_funded_by_investors     1
interest_rate               2
loan_length_Month           0
loan_purpose                0
debt_to_inc_ratio           1
State                       0
home_ownership              1
monthly_inc                 1
FICO_Range                  2
open_REDIT_lines            3
revolving_CREDIT_Bal        3
last_6month_inquiries       3
employment_Length          77
dtype: int64

In [10]:
loan_data.drop( columns='employment_Length' , inplace= True)
loan_data.dropna( inplace= True)

In [11]:
loan_data.shape

(2489, 13)

In [12]:
# seperate categorical and continuous variables
loan_data_conti_vars = loan_data.loc[:, (loan_data.dtypes == 'float64') | (loan_data.dtypes == 'int64')]
loan_data_cat_vars = loan_data.loc[:, (loan_data.dtypes == 'object')]

In [13]:
loan_data_conti_vars.apply(fn_descriptives)

,amt_req,amt_funded_by_investors,interest_rate,loan_length_Month,debt_to_inc_ratio,monthly_inc,open_REDIT_lines,revolving_CREDIT_Bal,last_6month_inquiries
dtype,float64,float64,float64,float64,float64,float64,float64,float64,float64
cardinality,378,707,273,2,1664,630,29,2342,10
n_tot,2489,2489,2489,2489,2489,2489,2489,2489,2489
n,2489,2489,2489,2489,2489,2489,2489,2489,2489
n_miss,0,0,0,0,0,0,0,0,0
n_miss_perc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sum,30888300.0,29879833.09,32533.5,102756.0,38286.26,14150580.53,25075.0,37883581.0,2254.0
var,60945301.903335,60039786.026045,17.450548,98.935762,56.357326,15727546.230929,20.3333,334804674.667375,1.517208
std,7806.747204,7748.534444,4.177385,9.946646,7.507152,3965.797049,4.509246,18297.668558,1.23175
mean,12409.923664,12004.754154,13.070912,41.28405,15.382186,5685.2473,10.074327,15220.40217,0.905585


In [14]:
# outlier treatment
for var in loan_data_conti_vars.columns:
        loan_data[var] = np.where(loan_data[var] > loan_data[var].quantile(0.99) ,
                                    loan_data[var].quantile(0.99), loan_data[var])

### a. Intrest rate is varied for different loan amounts (Less intrest charged for high loan amounts)

In [15]:
# Ho: No corelation, independant
# Ha: variables are related, dependant
   
# CI: 95%
# p: 0.05
  
# rejection rule: p is low null will go, p is high null will fly

# perform the test
stats.pearsonr( loan_data.interest_rate , loan_data.amt_funded_by_investors )

# business conclusion
# there is a direct corelation of the pre usage with latest month usage
# for the larger amount of loan interset rate is lower and vice- versa

(0.3341023116568378, 5.737570334459154e-66)

### b. Loan length is directly effecting intrest rate.

In [16]:
# Ho: No corelation, independant
# Ha: variables are related, dependant
   
# CI: 95%
# p: 0.05
  
# rejection rule: p is low null will go, p is high null will fly

# perform the test
stats.spearmanr( loan_data.interest_rate , loan_data.loan_length_Month )

# business conclusion
# there is a direct corelation of the pre usage with latest month usage
# for longer loan length, interset rate is low and vice- versa

SpearmanrResult(correlation=0.3897344194175745, pvalue=4.3727413630439584e-91)

### c. Inrest rate varies for different purpose of loans

In [17]:
loan_data.loan_purpose.unique()

array(['debt_consolidation', 'credit_card', 'other', 'moving', 'car',
       'vacation', 'home_improvement', 'house', 'major_purchase',
       'educational', 'medical', 'wedding', 'small_business',
       'renewable_energy'], dtype=object)

In [18]:
# data prep

# samples for each loan purposes
s1 = loan_data.loc[ loan_data.loan_purpose == 'debt_consolidation', 'interest_rate' ]
s2 = loan_data.loc[ loan_data.loan_purpose == 'credit_card', 'interest_rate' ]
s3 = loan_data.loc[ loan_data.loan_purpose == 'other', 'interest_rate' ]
s4 = loan_data.loc[ loan_data.loan_purpose == 'moving', 'interest_rate' ]
s5 = loan_data.loc[ loan_data.loan_purpose == 'car', 'interest_rate' ]
s6 = loan_data.loc[ loan_data.loan_purpose == 'vacation', 'interest_rate' ]
s7 = loan_data.loc[ loan_data.loan_purpose == 'home_improvement', 'interest_rate' ]
s8 = loan_data.loc[ loan_data.loan_purpose == 'house', 'interest_rate' ]
s9 = loan_data.loc[ loan_data.loan_purpose == 'major_purchase', 'interest_rate' ]
s10 = loan_data.loc[ loan_data.loan_purpose == 'educational', 'interest_rate' ]
s11 = loan_data.loc[ loan_data.loan_purpose == 'medical', 'interest_rate' ]
s12 = loan_data.loc[ loan_data.loan_purpose == 'wedding', 'interest_rate' ]
s13 = loan_data.loc[ loan_data.loan_purpose == 'small_business', 'interest_rate' ]
s14 = loan_data.loc[ loan_data.loan_purpose == 'renewable_energy', 'interest_rate' ]


In [19]:
# Ho: samples are from same population
# Ha: samples are from different population
   
# CI: 95%
# p: 0.05
  
# rejection rule: p is low null will go, p is high null will fly

# perform the test
print( stats.f_oneway( s1, s2, s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14 ) )

# Business conclusion
# Interest rate for different loan purpose is different.

F_onewayResult(statistic=7.485329679963486, pvalue=1.1463480725465721e-14)


### d. There is relationship between FICO scores and Home Ownership. It means that, People with owning home will have high FICO scores.

In [20]:
# data prep for chi square test
obs = pd.crosstab( loan_data.home_ownership, loan_data.FICO_Range )

# Ho: No relationship, independant
# Ha: variables are related, dependant
   
# CI: 95%
# p: 0.05
  
# rejection rule: p is low null will go, p is high null will fly

print ('p value =', stats.chi2_contingency( obs )[1])

# business conclusion
# people who owns a house has high FICO score as compare to people who doesn't.

p value = 8.424183189199129e-17


## BUSINESS PROBLEM - 2

In [21]:
price_quotes = pd.read_csv(data + '\\Price_Quotes.csv')
price_quotes.head()

,Order_Number,Barry_Price,Mary_Price
0,1,126,114
1,2,110,118
2,3,138,114
3,4,142,111
4,5,146,129


In [22]:
price_quotes.isnull().sum()

Order_Number    0
Barry_Price     0
Mary_Price      0
dtype: int64

### Q. We would like to assess if there is any difference in the average price quotes provided by Mary and Barry.

In [23]:
# print the avg/mean of the samples
print( 'mean of Barry price:', round(price_quotes.Barry_Price.mean(), 1) )
print( 'mean of Mary price:', round(price_quotes.Mary_Price.mean(), 1) )

mean of Barry price: 124.3
mean of Mary price: 114.8


In [24]:
# Ho: u2 = u1
# Ha: u2 <> u1

# CI: 95%
# p: 0.05
  
# rejection rule: p is low null will go, p is high null will fly

# perform the test
print( stats.ttest_rel( price_quotes.Barry_Price, price_quotes.Mary_Price ) )

# business conclusion
# Their is average price difference in Barry and Mary quotes.

Ttest_relResult(statistic=2.521376510892349, pvalue=0.02840588045242053)


## BUSINESS PROBLEM - 3

In [25]:
treatment_faci = pd.read_csv(data + '\\Treatment_Facility.csv')
treatment_faci.head(2)

,Month,Reengineer,Employee_Turnover,VAR4,VAR5
0,1,Prior,0.0000,24.390244,42.682927
1,2,Prior,6.0606,19.354839,25.806452


In [26]:
treatment_faci.rename( columns={'VAR4':'TRFF(%)','VAR5':'CI(%)'}, inplace=True)

In [27]:
treatment_faci.isnull().sum()

Month                0
Reengineer           0
Employee_Turnover    0
TRFF(%)              0
CI(%)                0
dtype: int64

### Q. Determine what effect, if any, the reengineering effort had on the incidence behavioral problems and staff turnover. i.e To determine if the reengineering effort changed the critical incidence rate. Is there evidence that the critical incidence rate improved?

In [28]:
treatment_faci.head(2)

,Month,Reengineer,Employee_Turnover,TRFF(%),CI(%)
0,1,Prior,0.0000,24.390244,42.682927
1,2,Prior,6.0606,19.354839,25.806452


In [29]:
#samples for each Reengineer 
s1 = treatment_faci.loc[ treatment_faci.Reengineer == 'Prior', 'CI(%)' ]
s2 = treatment_faci.loc[ treatment_faci.Reengineer == 'Post', 'CI(%)' ]

# print the avg/mean of three samples
print( 'mean of s1:', round(s1.mean(), 1), 
          '| mean of s2:', round(s2.mean(), 1))

mean of s1: 53.9 | mean of s2: 23.3


In [30]:
# Ho: u2 <= u1
# Ha: u2 > u1

# CI: 95%
# p: 0.05
  
# rejection rule: p is low null will go, p is high null will fly

# perform the test
print( stats.ttest_ind( s1, s2 ) )

# business conclusion
# WIth the give data we can not say that their is any effect on the reengineering efforts.

Ttest_indResult(statistic=1.6279144253528646, pvalue=0.12091989189884149)


## BUSINESS PROBLEM-4

In [31]:
priority_assessment = pd.read_csv(data + '\\Priority_Assessment.csv')
priority_assessment.head(5)

,Days,Priority
0,3.3,High
1,7.9,Medium
2,0.3,High
3,0.7,Medium
4,8.6,Medium


In [32]:
priority_assessment.isnull().sum()

Days        0
Priority    0
dtype: int64

### BUSINESS PROBLEM: We will focus on the prioritization system. If the system is working, then high priority jobs, on average, should be completed more quickly than medium priority jobs, and medium priority jobs should be completed more quickly than low priority jobs. Use the data provided to determine whether this is, in fact, occurring.

In [33]:
# Ho: No corelation, independant
# Ha: variables are related, dependant
   
# CI: 95%
# p: 0.05
  
# rejection rule: p is low null will go, p is high null will fly

# perform the test
stats.spearmanr( priority_assessment.Days , priority_assessment.Priority)

# business conclusion:
# With the given data we cannot say that the prioritization system is working.

SpearmanrResult(correlation=0.017913791551236326, pvalue=0.6505153120156153)

## BUSINESS PROBLEM-5

In [34]:
films = pd.read_csv(data + '\\Films.csv')
films.head(2)

,_rowstate_,Movie,Gender,Marital_Status,Sinage,Parking,Clean,Overall,Age,Income,Hear_About
0,0,Ferris Buellers Day Off,Female,Married,2.0,2.0,2.0,2.0,3.0,1.0,5
1,0,Ferris Buellers Day Off,Female,Single,1.0,1.0,1.0,1.0,2.0,1.0,5


In [35]:
films.isnull().sum()

_rowstate_         0
Movie              0
Gender             0
Marital_Status     2
Sinage             2
Parking            2
Clean              3
Overall            2
Age                2
Income            16
Hear_About         7
dtype: int64

In [36]:
for var in films.columns:
    films[var].fillna(value = films[var].mode()[0], inplace=True)

In [37]:
def gender (x):
    if x.Gender == 'Male' :
        return '1'
    elif x.Gender == 'Female'  :
        return '0'
    elif x.Gender == '1'  :
        return '1'
    else :
        return '0'

In [38]:
#converting males to 1 and females to 0 
films.Gender = films.apply(gender, axis=1)

In [39]:
def married_status (x):
    if x.Marital_Status == 'Married' :
        return '1'
    elif x.Marital_Status == 'Single'  :
        return '0'
    elif x.Marital_Status == '1'  :
        return '1'
    else :
        return '0'

In [40]:
#converting married to 1 and singles to 0
films.Marital_Status = films.apply(married_status, axis=1)

### Q.1 What is the overall level of customer satisfaction?

In [41]:
overall = films.groupby('Overall')[['Overall']].count()
overall = overall.rename( columns={'Overall': 'count'}).reset_index()
overall =overall.rename( columns={'Overall': 'Satisfaction level'})
overall['Prob'] = overall['count'] / overall['count'].sum()
print(overall)
print('  As Satisfaction level 1 is Excellent and 2 is Good. That means their is 94% chance customers are satisfied. ')

   Satisfaction level  count      Prob
0                 1.0    151  0.457576
1                 2.0    162  0.490909
2                 3.0     12  0.036364
3                 4.0      1  0.003030
4                 5.0      4  0.012121
  As Satisfaction level 1 is Excellent and 2 is Good. That means their is 94% chance customers are satisfied. 


### Q.2 What factors are linked to satisfaction?

In [42]:
# data prep for chi square test
obs = pd.crosstab( films.Sinage,films.Overall )

# Ho: No relationship, independant
# Ha: variables are related, dependant
   
# CI: 95%
# p: 0.05
  
# rejection rule: p is low null will go, p is high null will fly

# perform the test

stats.chi2_contingency( obs )

# business conclusion
# with increase in sinage rating overall staifaction is also increases

(125.61859671027881,
 4.53298970003405e-19,
 16,
 array([[5.17060606e+01, 5.54727273e+01, 4.10909091e+00, 3.42424242e-01,
         1.36969697e+00],
        [7.22969697e+01, 7.75636364e+01, 5.74545455e+00, 4.78787879e-01,
         1.91515152e+00],
        [2.19636364e+01, 2.35636364e+01, 1.74545455e+00, 1.45454545e-01,
         5.81818182e-01],
        [2.74545455e+00, 2.94545455e+00, 2.18181818e-01, 1.81818182e-02,
         7.27272727e-02],
        [2.28787879e+00, 2.45454545e+00, 1.81818182e-01, 1.51515152e-02,
         6.06060606e-02]]))

In [43]:
# data prep for chi square test
obs = pd.crosstab( films.Parking,films.Overall )

# Ho: No relationship, independant
# Ha: variables are related, dependant
   
# CI: 95%
# p: 0.05
  
# rejection rule: p is low null will go, p is high null will fly

# perform the test

stats.chi2_contingency( obs )

# business conclusion
# with increase in parking rating overall staifaction is also increases

(230.61691663391676,
 4.783162283608494e-40,
 16,
 array([[6.17727273e+01, 6.62727273e+01, 4.90909091e+00, 4.09090909e-01,
         1.63636364e+00],
        [7.09242424e+01, 7.60909091e+01, 5.63636364e+00, 4.69696970e-01,
         1.87878788e+00],
        [1.32696970e+01, 1.42363636e+01, 1.05454545e+00, 8.78787879e-02,
         3.51515152e-01],
        [3.20303030e+00, 3.43636364e+00, 2.54545455e-01, 2.12121212e-02,
         8.48484848e-02],
        [1.83030303e+00, 1.96363636e+00, 1.45454545e-01, 1.21212121e-02,
         4.84848485e-02]]))

In [44]:
# data prep for chi square test
obs = pd.crosstab( films.Clean,films.Overall )

# Ho: No relationship, independant
# Ha: variables are related, dependant
   
# CI: 95%
# p: 0.05
  
# rejection rule: p is low null will go, p is high null will fly

# perform the test

stats.chi2_contingency( obs )

# business conclusion
# with increase in clean rating overall staifaction is also increases

(127.71945555999517,
 1.7772535915434025e-19,
 16,
 array([[6.58909091e+01, 7.06909091e+01, 5.23636364e+00, 4.36363636e-01,
         1.74545455e+00],
        [7.09242424e+01, 7.60909091e+01, 5.63636364e+00, 4.69696970e-01,
         1.87878788e+00],
        [1.09818182e+01, 1.17818182e+01, 8.72727273e-01, 7.27272727e-02,
         2.90909091e-01],
        [2.28787879e+00, 2.45454545e+00, 1.81818182e-01, 1.51515152e-02,
         6.06060606e-02],
        [9.15151515e-01, 9.81818182e-01, 7.27272727e-02, 6.06060606e-03,
         2.42424242e-02]]))

### Q.3 What is the demographic profile of Film on the Rocks patrons?

In [45]:
# Demography on Gender
gender = films.groupby('Gender')[['Gender']].count()
gender = gender.rename( columns={'Gender': 'count'}).reset_index()
gender['Prob'] = gender['count'] / gender['count'].sum()
print(gender)
print('  AS Gender 0 is Female and 1 is male. That means their is 64% probability audiance is female')

  Gender  count      Prob
0      0    213  0.645455
1      1    117  0.354545
  AS Gender 0 is Female and 1 is male. That means their is 64% probability audiance is female


In [46]:
#Demography on Martial Status
Marital_Status = films.groupby('Marital_Status')[['Marital_Status']].count()
Marital_Status = Marital_Status.rename( columns={'Marital_Status': 'count'}).reset_index()
Marital_Status['Prob'] = Marital_Status['count'] / Marital_Status['count'].sum()
print(Marital_Status)
print('  Their is 69% probability that the audiance is Married')

  Marital_Status  count     Prob
0              0    230  0.69697
1              1    100  0.30303
  Their is 69% probability that the audiance is Married


In [47]:
#Demography on Martial Status
Age = films.groupby('Age')[['Age']].count()
Age = Age.rename( columns={'Age': 'count'}).reset_index()
Age['Prob'] = Age['count'] / Age['count'].sum()
print(Age)
print('  Their is 53% probability that the age of the audiance is between 13-30')

   Age  count      Prob
0  1.0     26  0.078788
1  2.0    177  0.536364
2  3.0    117  0.354545
3  4.0     10  0.030303
  Their is 53% probability that the age of the audiance is between 13-30


In [48]:
#Demography on Income
Income = films.groupby('Income')[['Income']].count()
Income = Income.rename( columns={'Income': 'count'}).reset_index()
Income['Prob'] = Income['count'] / Income['count'].sum()
print(Income)
print('  Their is 47% probability that the Income of the audiance is less than 50K')

   Income  count      Prob
0     1.0    158  0.478788
1     2.0     82  0.248485
2     3.0     90  0.272727
  Their is 47% probability that the Income of the audiance is less than 50K


### Q.4 In what media outlet(s) should the film series be advertised?

In [49]:
#Demography on Income
Hear_About = films.groupby('Hear_About')[['Hear_About']].count()
Hear_About = Hear_About.rename( columns={'Hear_About': 'count'}).reset_index()
Hear_About['Prob'] = Hear_About['count'] / Hear_About['count'].sum()
print(Hear_About.sort_values(by='Prob', ascending = False))
print('  Their is 70% probability that the audiance heard about the film series solely through word of mouth.')

   Hear_About  count      Prob
9           5    233  0.706061
7           4     41  0.124242
0           1     22  0.066667
4           3     14  0.042424
2           2     12  0.036364
3         2,5      2  0.006061
5         3,4      2  0.006061
1         1,5      1  0.003030
6         3,5      1  0.003030
8         4,5      1  0.003030
10        5,4      1  0.003030
  Their is 70% probability that the audiance heard about the film series solely through word of mouth.
